In [12]:
from music21 import *
from collections import Counter, defaultdict

import collections
import numpy as np
import pandas as pd
import gzip
import random
import nltk
import music21
import util


In [ ]:
us = environment.UserSettings()
us['musicxmlPath'] = 'C:/Program Files/MuseScore 3/bin/MuseScore3.exe'
us['musescoreDirectPNGPath'] = 'C:/Program Files/MuseScore 3/bin/MuseScore3.exe'

In [ ]:
coreCorpus = music21.corpus.corpora.CoreCorpus()
curr_corpus = music21.corpus.search('mozart', 'composer')

scores = []
for c in curr_corpus:
    score = c.parse()
    scores.append(score)

In [ ]:
# Add lakh corpus
localCorpus = corpus.corpora.LocalCorpus()
lakh_corpus = corpus.corpora.LocalCorpus('lakh')
print(lakh_corpus.existsInSettings)
lakh_corpus.addPath('./midi/lakh')
lakh_corpus.save()
print(lakh_corpus.existsInSettings)

In [19]:
lakh_corpus = corpus.corpora.LocalCorpus('lakh')
scores = []
lakh_subset = lakh_corpus.metadataBundle[0:10]
for c in lakh_subset:
    score = c.parse()
    score = util.transpose_piece(score, 'C')
    scores.append(c.parse())

In [23]:
len(scores[0].parts

17

In [ ]:
note.Note('B').isChord

In [ ]:
test = scores[0].parts[0].getElementsByClass(stream.Measure).flat.notesAndRests.stream()
for i in test:
    if i.isChord:
        print(i)

In [20]:
noteIterator = scores[0].parts[0].getElementsByClass(stream.Measure)
all_intervals_per_measure = []
for i in noteIterator:
    firstNote = None
    counter = 1
    m = []
    for j in i.notesAndRests:
        if j.isChord or j.isRest or j is None:
            continue
        if counter == 1:
            firstNote = j
            counter += 1
            continue
        #print(j.beatStr)
        interv = interval.Interval(firstNote,  j).name
        m.append(interv)
        counter += 1
    all_intervals_per_measure.append(interv)
all_intervals_per_measure

[]

In [ ]:
bigrams = list(nltk.bigrams(all_intervals_per_measure))
bigrams

In [ ]:
bigrams = list(nltk.bigrams(all_intervals_per_measure))
matrix = defaultdict(lambda: defaultdict(lambda: 0))

for i1,i2 in bigrams:
    matrix[i1][i2] += 1
    
for i1 in matrix:
    total = float(sum(matrix[i1].values()))
    for i2 in matrix[i1]:
        matrix[i1][i2] /= total
    
pd_matrix = pd.DataFrame(matrix)
pd_matrix = pd_matrix.fillna(0)
pd_matrix

In [ ]:
# Get a part of the piece
noteIterator = scores[0].parts[0].getElementsByClass(stream.Measure).flat.notesAndRests
allNotes = []

for el in noteIterator:
    if el.isChord:
        continue
    if el.isRest:
        noteName = 'REST'
    elif('-' in el.nameWithOctave):
        noteName = el.pitch.getEnharmonic().nameWithOctave
    else:
        noteName = el.nameWithOctave
    allNotes.append(noteName)
    

In [ ]:
allNotes

In [ ]:
trigrams = list(nltk.trigrams(allNotes))
possibleNotes = set(trigrams)

In [ ]:
possibleNotes

In [ ]:
zeros = np.full((len(possibleNotes), len(possibleNotes)), 0)
matrix = pd.DataFrame(zeros, index=possibleNotes, columns=possibleNotes)
matrix = matrix.astype(float)

In [ ]:
trigrams

In [ ]:
zeros = np.full((len(possibleNotes), len(possibleNotes)), 0)
matrix = defaultdict(lambda: defaultdict(lambda: 0))

for n1,n2,n3 in trigrams:
    matrix[(n1,n2)][n3] += 1
    #     if (n1,n2) in matrix:
#         if n3 in matrix[(n1,n2)]:
#             matrix[(n1, n2)][n3] = matrix[(n1, n2)][n3] + 1
#         else:
#             matrix[(n1,n2)][n3] = 1
#     else:
#         matrix[(n1,n2)] = {}
#         matrix[(n1,n2)][n3] = 1


for n1_n2 in matrix:
    total_count = float(sum(matrix[n1_n2].values()))
    for n3 in matrix[n1_n2]:
        matrix[n1_n2][n3] /= total_count
        
pd_matrix = pd.DataFrame(matrix)
pd_matrix = pd_matrix.fillna(0)
pd_matrix

In [ ]:
pd_matrix.to_csv('./test.csvv')

In [ ]:
# Get frequency array
counter = collections.Counter(flatten(bigrams))
# Initial note counter
counter[' '] = len(allNotesPerPiece)

noteRows = possibleNotes.add(' ')

zeros = np.full((len(possibleNotes), len(possibleNotes)), 0)
matrix = pd.DataFrame(zeros, index=possibleNotes, columns=possibleNotes)
matrix = matrix.astype(float)

for allNotes in allNotesPerPiece:
    # Fill transition matrix frequencies
    for i in range(len(allNotes)+1):
        # First note
        if(i == 0):
            matrix[' '][allNotes[i]] = matrix[' '][allNotes[i]] + 1
            continue
        # Last note
        if(i == len(allNotes)):
            matrix[allNotes[i-1]][' '] = matrix[allNotes[i-1]][' '] + 1
            continue

        currNote = allNotes[i-1]
        nextNote = allNotes[i]

        matrix[currNote][nextNote] = matrix[currNote][nextNote] + 1
